Downloading the [Zip file](http://www.manythings.org/anki/fra-eng.zip)

In [5]:
!wget http://www.manythings.org/anki/fra-eng.zip

--2022-03-11 12:21:37--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.92.44, 172.67.186.54
Connecting to www.manythings.org (www.manythings.org)|104.21.92.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6532197 (6.2M) [application/zip]
Saving to: 'fra-eng.zip'

     0K .......... .......... .......... .......... ..........  0% 1.19M 5s
    50K .......... .......... .......... .......... ..........  1% 10.4M 3s
   100K .......... .......... .......... .......... ..........  2%  805K 4s
   150K .......... .......... .......... .......... ..........  3%  150K 14s
   200K .......... .......... .......... .......... ..........  3% 8.57M 11s
   250K .......... .......... .......... .......... ..........  4% 4.94M 9s
   300K .......... .......... .......... .......... ..........  5% 10.2M 8s
   350K .......... .......... .......... .......... ..........  6% 6.01M 7s
   400K .......... .......... .......... .

**Extracting the Zip file**

In [6]:
import zipfile
zip = zipfile.ZipFile('fra-eng.zip')
zip.extractall()

**Dependencies**

In [7]:
import string,re
from unicodedata import normalize
from numpy import array,argmax
from pickle import load,dump
from numpy.random import rand,shuffle

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential,load_model
from keras.layers import LSTM,Dense,Embedding,RepeatVector,TimeDistributed
from nltk.translate.bleu_score import SmoothingFunction,corpus_bleu
smoothie = SmoothingFunction().method4

**Loading the file and reading the content of the file**

In [10]:
# load file into memory
def load_file(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

**Splitting the sentence into pairs**

In [11]:
# split a loaded document into sentences
def splitting_sentence(doc):
	sentences = doc.strip().split('\n')
	pairs = [sentence.split('\t') for sentence in  sentences]
	return pairs

**Cleaning the pairs**

In [12]:
# cleaning a list of sentences and creating pairs

def clean_pairs(sentences):
	cleaned = list()
 
	# preparing regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))

	# preparing translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)

  # iterating over each pair
	for pair in sentences:
		clean_pair = list()
  
		for sentence in pair:
			# normalizing unicode characters
			sentence = normalize('NFD', sentence).encode('ascii', 'ignore')
			sentence = sentence.decode('UTF-8')
			# tokenizing on white space
			sentence = sentence.split()
			# converting to lowercase
			sentence = [word.lower() for word in sentence]
			# removing punctuation from each token
			sentence = [word.translate(table) for word in sentence]
			# removing non-printable chars form each token
			sentence = [re_print.sub('', w) for w in sentence]
			# removing tokens with numbers in them
			sentence = [word for word in sentence if word.isalpha()]
			# storing as string
			clean_pair.append(' '.join(sentence))
		cleaned.append(clean_pair)
	return array(cleaned)

**Saving the Cleaned data**

In [13]:
def saving_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print(filename,': Saved')

**Saving data in .pkl format**

In [14]:
# load dataset

filename = 'fra.txt'
doc = load_file(filename)

# split into english-french pairs
pairs = splitting_sentence(doc)

# clean sentences
clean_pairs = clean_pairs(pairs)

# save clean pairs to file
saving_clean_data(clean_pairs, 'english-french.pkl')

print('English','-->',"French")
# spot check
for i in range(25):
	print(clean_pairs[i,0],'-->',clean_pairs[i,1])

english-french.pkl : Saved
English --> French
go --> va
go --> marche
go --> bouge
hi --> salut
hi --> salut
run --> cours
run --> courez
run --> prenez vos jambes a vos cous
run --> file
run --> filez
run --> cours
run --> fuyez
run --> fuyons
run --> cours
run --> courez
run --> prenez vos jambes a vos cous
run --> file
run --> filez
run --> cours
run --> fuyez
run --> fuyons
who --> qui
wow --> ca alors
duck --> a terre
duck --> baissetoi


**Loading the cleaned data**

In [15]:
# load a clean dataset
def loading_cleaned_data(filename):
	return load(open(filename, 'rb'))

In [16]:
# load dataset
data = loading_cleaned_data('english-french.pkl')
print(data.shape) 

(192341, 3)


**Scaling of data** 

**Size**

1.Dataset - 20000

2.Training - 18000

3.Testing - 2000   



In [17]:
# reducing dataset size (scaling) 

new_data_size = 20000
dataset = data[:new_data_size, :]

# randomly shuffling the dataset to get proper training and testing data
shuffle(dataset)

# splitting into training and testing (90%-10%)
train, test = dataset[:18000], dataset[18000:]

# saving the cleaned data,train data and test data 
saving_clean_data(dataset, 'english-french-both.pkl')
saving_clean_data(train, 'english-french-train.pkl')
saving_clean_data(test, 'english-french-test.pkl')

english-french-both.pkl : Saved
english-french-train.pkl : Saved
english-french-test.pkl : Saved


In [18]:
# loading datasets and saving it into variables
dataset = loading_cleaned_data('english-french-both.pkl')
train = loading_cleaned_data('english-french-train.pkl')
test = loading_cleaned_data('english-french-test.pkl')

**Creating a tokenizer for the lines and finding the maximum length phrase**

In [19]:
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

**Size of English & French vocabulary and their max phrase length**

In [20]:
# preparing the english tokenizer

eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])

print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

# preparing the french tokenizer

fra_tokenizer = create_tokenizer(dataset[:, 1])
fra_vocab_size = len(fra_tokenizer.word_index) + 1
fra_length = max_length(dataset[:, 1])
print('French Vocabulary Size: %d' % fra_vocab_size)
print('French Max Length: %d' % (fra_length))


English Vocabulary Size: 3418
English Max Length: 5
French Vocabulary Size: 6977
French Max Length: 11


**Encoding to integers and padding to the maximum phrase length**

In [21]:
# Input and Output sequence must be encoded to integers and padded to the maximum phrase length
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	x = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	x = pad_sequences(x, maxlen=length, padding='post')
	return x

# One hot encoding to max phrase length
def one_hot_encoding(sequences, vocab_size):
	y_1 = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		y_1.append(encoded)
	y = array(y_1)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

**Training and Testing Data**

In [22]:
# preparing training data
trainX = encode_sequences(fra_tokenizer, fra_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = one_hot_encoding(trainY, eng_vocab_size)

# prepare testing data
testX = encode_sequences(fra_tokenizer, fra_length, test[:, 1])
testY = encode_sequences(eng_tokenizer,eng_length, test[:, 0])
testY = one_hot_encoding(testY, eng_vocab_size)

In [23]:
print('training size:',trainX.shape,trainY.shape)
print('testing size:',testX.shape,testY.shape)

training size: (18000, 11) (18000, 5, 3418)
testing size: (2000, 11) (2000, 5, 3418)


**Building the model**

In [24]:
def model_building(source_vocab, target_vocab, source_len, target_len, units):
	model = Sequential()
	model.add(Embedding(source_vocab, units, input_length=source_len, mask_zero=True))
	model.add(LSTM(units))
	model.add(RepeatVector(target_len))
	model.add(LSTM(units, return_sequences=True))
	model.add(TimeDistributed(Dense(target_vocab, activation='softmax')))
	return model

**Defining and Compiling the model**

In [25]:
model = model_building(fra_vocab_size, eng_vocab_size, fra_length, eng_length, 512)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['acc'])

**Model Summary**

In [26]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 512)           3572224   
                                                                 
 lstm (LSTM)                 (None, 512)               2099200   
                                                                 
 repeat_vector (RepeatVector  (None, 5, 512)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 5, 512)            2099200   
                                                                 
 time_distributed (TimeDistr  (None, 5, 3418)          1753434   
 ibuted)                                                         
                                                                 
Total params: 9,524,058
Trainable params: 9,524,058
Non-

In [27]:
# Stop model if accuracy of the model doesn't changes by more than 0.01 
# Patience = 5 : After each 5 epochs if no improvement is there then training will be stopped.
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_acc',patience= 5,min_delta=0.01)

**Fitting the model**

1.Epochs = 50

2.Batch_size = 25

In [28]:
# fit model
model.fit(trainX, trainY, epochs= 50, batch_size=25, validation_data=(testX, testY), verbose=2,callbacks=[es])

Epoch 1/50
720/720 - 238s - loss: 3.6208 - acc: 0.4608 - val_loss: 3.1023 - val_acc: 0.5105 - 238s/epoch - 330ms/step
Epoch 2/50
720/720 - 248s - loss: 2.7518 - acc: 0.5509 - val_loss: 2.5777 - val_acc: 0.5802 - 248s/epoch - 345ms/step
Epoch 3/50
720/720 - 213s - loss: 2.1855 - acc: 0.6101 - val_loss: 2.2388 - val_acc: 0.6229 - 213s/epoch - 295ms/step
Epoch 4/50
720/720 - 219s - loss: 1.7404 - acc: 0.6578 - val_loss: 2.0150 - val_acc: 0.6485 - 219s/epoch - 305ms/step
Epoch 5/50
720/720 - 217s - loss: 1.3641 - acc: 0.7049 - val_loss: 1.8703 - val_acc: 0.6700 - 217s/epoch - 302ms/step
Epoch 6/50
720/720 - 219s - loss: 1.0443 - acc: 0.7575 - val_loss: 1.7843 - val_acc: 0.6859 - 219s/epoch - 305ms/step
Epoch 7/50
720/720 - 197s - loss: 0.7882 - acc: 0.8055 - val_loss: 1.6957 - val_acc: 0.7106 - 197s/epoch - 273ms/step
Epoch 8/50
720/720 - 201s - loss: 0.5908 - acc: 0.8479 - val_loss: 1.6537 - val_acc: 0.7194 - 201s/epoch - 279ms/step
Epoch 9/50
720/720 - 214s - loss: 0.4492 - acc: 0.8802 -

**Evaluating model and calculating BLEU Score**

Evaluation involves two steps: 

1.Generating a translated output sequence, and 

2.then repeating this process for many input examples and summarizing the skill of the model across multiple cases.

In [29]:
# mapping integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [30]:
# generating target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [31]:

# evaluating the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
  
  # Creating empty lists for actual phrases(French) and predicted phrases(English) 
  actual,predicted = list(),list()
  a,b,c = list(),list(),list()
  for i,source in enumerate(sources):

    # reshaping to the required size
    source = source.reshape((1, source.shape[0]))

    # predicting for the english tokenizer
    translation = predict_sequence(model, eng_tokenizer, source)
    # raw_dataset = raw_dataset[i].split(' ') 
    # print(raw_dataset[i][1])

    raw_src,raw_target = raw_dataset[i][1],raw_dataset[i][0]
    
    # First 10 Predictions
    if i <= 10:
      print('source = ',raw_src,'<--->', ' target = ',raw_target,'<--->','  predicted = ',translation)

    actual.append([raw_target.split()])
    predicted.append(translation.split())
  
  # calculating BLEU score
  print('-------------------------------------------')
  print('BLEU Score :')
  print('BLEU score-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0),smoothing_function=smoothie,auto_reweigh=False))
  print('BLEU score-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=smoothie,auto_reweigh=False))

**Evaluating Model on training data**

In [32]:
evaluate_model(model,eng_tokenizer,trainX,train)

source =  jadore les enigmes <--->  target =  i love puzzles <--->   predicted =  i love puzzles
source =  la radio est allumee <--->  target =  the radio is on <--->   predicted =  the radio is on
source =  tom a ete cruel <--->  target =  tom was cruel <--->   predicted =  tom was cruel
source =  ne sois pas si lent <--->  target =  dont be so slow <--->   predicted =  dont be so slow
source =  jen veux une aussi <--->  target =  i want one too <--->   predicted =  i want one too
source =  ils sont pieges <--->  target =  theyre trapped <--->   predicted =  theyre trapped
source =  tom aida <--->  target =  tom helped <--->   predicted =  tom helped
source =  je me suis blesse <--->  target =  i got hurt <--->   predicted =  i got hurt
source =  il deteste courir <--->  target =  he hates running <--->   predicted =  he hates running
source =  tout est faux <--->  target =  its all wrong <--->   predicted =  its all wrong
source =  combien cela coutetil <--->  target =  how much is t

**Evaluating Model on testing data**

In [33]:
evaluate_model(model, eng_tokenizer, testX, test)

source =  ne touche pas a ceci <--->  target =  dont touch this <--->   predicted =  dont touch that
source =  tom peut parler <--->  target =  tom can talk <--->   predicted =  tom can talk
source =  je ne suis pas normale <--->  target =  im not normal <--->   predicted =  im not normal
source =  verrouillez le coffrefort <--->  target =  lock the safe <--->   predicted =  lock the safe
source =  jai ete secouee <--->  target =  i was shaken <--->   predicted =  i was attacked
source =  je pourrais etre le prochain <--->  target =  i could be next <--->   predicted =  i could feel well
source =  quand cela sestil termine <--->  target =  when did it end <--->   predicted =  when does it
source =  souriez je vous prie <--->  target =  please smile <--->   predicted =  please please
source =  je suis plus rapide <--->  target =  im faster <--->   predicted =  im am
source =  tom est deteste <--->  target =  tom is hated <--->   predicted =  tom is
source =  tout le monde vatil bien <--

In [8]:
import keras as ks
import tensorflow as tf

In [ ]:
keras_file = "translator.h5"
tf.keras.models.save_model(model,keras_file)
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()

In [ ]:
open("translator.tflite","wb").write(tfmodel)